# Data Preparation
## Data Description
The dataset (which is located in the `./data/` folder) contains the following relations:
* Relation `account` (containing 4500 records) where each record describes an account
* Relation `client` (containing 5369 records) where each record describes a client
* Relation `disposition` (containing 5369 records) where each record links a client to an account and indicates the rights of clients to operate accounts
* Relation `permanent order` (containing 6471 records) where each record describes a payment order
* Relation `transaction` (containing 1056320 records) where each record describes a transaction in an account
* Relation `loan` (containing 682 records) where each record describes a loan granted for a given account
* Relation `credit card` (containing 892 records) where each record describes a credit card issued to an account
* Relation `demographic data` (containing 77 records) where each record describes the demography of a district

### Relation `account`
Each account has static (e.g. date of creation, address of branch) and dynamic attributes (e.g. transactions and balances) which are given in the `permanent order` and `transaction` relations. This relation follows the schema:
| Attribute | Description |
| --------- | ----------- |
| account_id **int** | Account identifier |
| district_id **int** | Identifier of the district where the account is located |
| date **int** | Account creation date (format: YYMMDD) |
| frequency **string** | Frequency of issuance of statements |

Located in `./data/account.csv`

### Relation `client`
This relation contains attributes of the people who can manipulate accounts. One client can have multiple accounts and multiple clients can manipulate the same account.
| Attribute | Description |
| --------- | ----------- |
| client_id **int** | Client identifier |
| birth_number **string** | Birthday and sex (format: YYMMDD - in women, the month is incremented by 50), autoconverted to **int** by pandas |
| district_id **int** | Identifier of the district where the client lives |

Located in `./data/client.csv`

### Relation `disposition`
This relation links clients to accounts and includes the rights of clients to operate accounts.
| Attribute | Description |
| --------- | ----------- |
| disp_id **int** | Disposition identifier |
| client_id **int** | Client Identifier |
| account_id **int** | Account Identifier |
| type **string** | Disposition type (*owner*/*user*), only the owner can issue permanent orders and borrow money |

Located in `./data/disp.csv`

### Relation `permanent order`
| Attribute | Description |
| --------- | ----------- |
| order_id **int** | Order identifier |
| account_id **int** | Account identifier |
| bank_to | Recipient's bank (each bank has a unique two-letter code) |
| account_to | Recipient's account identifier |
| amount **int** | Debited amount |
| k_symbol | Characterization of the payment |

Not located in the `./data/` folder. Can it be built from the other relations?

### Relation `transaction`
| Attribute | Description |
| --------- | ----------- |
| trans_id | Transaction identifier |
| account_id | Account identifer |
| date | Transaction date (format: YYMMDD) |
| type | +/- transaction |
| operation | Transaction mode | 	 
| amount | Amount of money transacted |
| balance |	Balance after transaction |
| k_symbol | Characterization of the transaction |
| bank | Bank of the partner (each bank has unique two-letter code) |
| account | Account of the partner |

Located in `./data/trans_train.csv` and `./data/trans_test.csv` as training and test data for the model.

### Relation `loan`
Describes a service provided by the bank to its clients. Only one loan can be granted for an account.
| Attribute | Description |
| --------- | ----------- |
| loan_id | Loan identifier |
| account_id | Account identifier |
| date | Date when the loan was granted (format: YYMMDD) |
| amount | Amount of money borrowed |
| duration | Loan duration |
| payments | Number of monthly payments made on the loan | 	 
| status | Status of the loan payment ('A' stands for contract finished, no problems, 'B' stands for contract finished, loan not paid, 'C' stands for running contract, OK so far, 'D' stands for running contract, client in debt) |

Located in `./data/loan_train.csv` and `./data/loan_test.csv` as training and test data for the model.

### Relation `credit card`
Describes a service provided by the bank to its clients. Multiple credit cards can be issued for the same account.
| Attribute | Description |
| --------- | ----------- |
| card_id | Card identifier |
| disp_id | Disposition to an account |
| type **string** | Type of card (possible values are "junior", "classic", "gold") |
| issued | Issue date in the form YYMMDD |

Located in `./data/card_train.csv` and `./data/card_test.csv` as training and test data for the model.

### Relation `demographic data`
Contains publicly available information about the districts, additional information about the clients can be deduced from this.
| Attribute | Description |
| --------- | ----------- |
| A1 | District Identifier |
| A2 | District name |
| A3 | Region | 
| A4 | No. of inhabitants | |
| A5 | No. of municipalities with inhabitants < 499 |
| A6 | No. of municipalities with inhabitants 500-1999 |
| A7 | No. of municipalities with inhabitants 2000-9999 |
| A8 | No. of municipalities with inhabitants >10000 |
| A9 | No. of cities |
| A10 |	Ratio of urban inhabitants |
| A11 |	Average salary |
| A12 |	Unemployment rate '95 |
| A13 |	Unemployment rate '96 |
| A14 |	No. of enterpreneurs per 1000 inhabitants |
| A15 |	No. of commited crimes '95 |
| A16 |	No. of commited crimes '96 |

## Data Processing
Load Model class which will help us read the datasets into DataFrames.

In [1]:
from model.model import Model

model = Model()

In [2]:
# Load Dataframes
account_df = model.get_accounts()
client_df = model.get_clients()
# card_test_df = model.get_cards('test')
# card_train_df = model.get_cards('train')
disp_df = model.get_disps()
district_df = model.get_districts()
loan_test_df = model.get_loans('test')
loan_train_df = model.get_loans('train')
trans_test_df = model.get_transactions('test')
trans_train_df = model.get_transactions('train')

Merge all datasets and prepare a dataset for training and another for testing taking into account the files ending in ``_train`` and ``_test``.

In [3]:
def merge_all():
    df = disp_df.merge(client_df, on='client_id')
    df = df.merge(district_df, on='district_id')
    df['date'] = df.merge(account_df, on='account_id')['date']
    return df

def merge_train(df):
    df = df.merge(loan_train_df, on='account_id')
    df = df.merge(trans_train_df, on='account_id')
    return df

def merge_test(df):
    df = df.merge(loan_test_df, on='account_id')
    df = df.merge(trans_test_df, on='account_id')
    return df 

df = merge_all()
train_df = merge_train(df)
test_df = merge_test(df)
train_df.head()

df = merge_all()
train_df = merge_train(df)
test_df = merge_test(df)
train_df.head()

,disp_id,client_id,account_id,type,birth_number,district_id,gender,age,district_name,region,...,loan_amount,duration,payments,status,trans_id,trans_date,trans_type,operation,trans_amount,balance
0,13182,13490,10973,OWNER,1969-05-25,18,0,52,Pisek,south Bohemia,...,154416,48,3217,1,3302598,1993-04-20,credit,credit in cash,8897.0,8897.0
1,13182,13490,10973,OWNER,1969-05-25,18,0,52,Pisek,south Bohemia,...,154416,48,3217,1,3302582,1993-04-20,credit,credit in cash,400.0,9297.0
2,13182,13490,10973,OWNER,1969-05-25,18,0,52,Pisek,south Bohemia,...,154416,48,3217,1,3526454,1993-04-30,credit,interest credited,13.6,9310.6
3,13182,13490,10973,OWNER,1969-05-25,18,0,52,Pisek,south Bohemia,...,154416,48,3217,1,3302588,1993-05-03,credit,credit in cash,25724.0,35034.6
4,13182,13490,10973,OWNER,1969-05-25,18,0,52,Pisek,south Bohemia,...,154416,48,3217,1,3302586,1993-05-15,credit,credit in cash,25060.0,60094.6


After merging the DataFrame containing test data has 30200 rows and 37 columns (features).

In [4]:
test_df.shape

(30200, 37)

Now we need to save the test and train datasets before aggregating because these tables are useful for the descriptive task.

In [5]:
train_df.to_csv('train_unagg.csv', index=False)
test_df.to_csv('test_unagg.csv', index=False)

## Data Aggregation

We are now going to aggregate data on the test and train datasets and create new features from the values in those datasets.
The aggregation operation creates new features representing statistics about:
* Transaction date: minimum (oldest transaction), maximum (latest transaction) and interval between transactions;
* Transaction type: counting of the various transaction types in the dataset, mean and standard deviation of transactions of a given type;
* Transaction amount: mean, minimum, maximum and standard deviation;
* Operation type: counting of the various operation types in the dataset, mean and standard deviation of operations of a given type.

The aggregation also creates features representing:
* The number of days since the last transaction;
* The ratio between the last balance and the amount borrowed;
* The ratio between the maximum balance and the amount borrowed;
* The number of days between the oldest and latest transaction in months;
* The average balance per month;
* The average amount in transactions in each month;
* The age of the borrower at the start of the loan;
* The age of the account in months.

After aggregating, the train dataset has 384 rows and 73 columns (a total of 36 new columns was created).

In [7]:
import numpy as np
from agg import *

def agg_features(df):
    agg_columns = ['loan_id', 'account_id', 'loan_date', 'loan_amount',
                 'duration', 'payments', 'status',
                 'birth_number',
                 'district_id', 'gender',
                 'no. of inhabitants',
                 'small_munis_rate', 'medium_munis_rate',
                 'large_munis_rate', 'larger_munis_rate',
                 'inhabitant_rate', 'no. of cities ', 
                 'ratio of urban inhabitants ',
                 'average salary ', 'unemploymant rate \'95 ',
                 'unemploymant rate \'96 ',
                 'no. of enterpreneurs per 1000 inhabitants ',
                 'crime_rate \'95',
                 'crime_rate \'96', 'date']

    df = df.groupby(agg_columns, as_index=False, group_keys=False).agg({
        'trans_date': ['max', 'min', days_between],
        'trans_amount': ['mean', 'min', 'max', 'std', 'last'],
        'operation': ['count', 
                    count_credit_op, count_collection_op, count_withdrawal_op, count_remittance_op, count_ccw_op, count_interest_op,
                    mean_credit_op, mean_collection_op, mean_withdrawal_op, mean_remittance_op, mean_ccw_op, mean_interest_op,
                    std_credit_op, std_collection_op, std_withdrawal_op, std_remittance_op, std_ccw_op, std_interest_op],
        'balance': ['mean', 'min', 'max', 'std', 'last', bal_range, bal_min],
        'trans_type': [count_withdrawal, count_credit, mean_withdrawal, mean_credit, std_withdrawal, std_credit]
    })

    df.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]

    df['days_last_trans'] = (df['loan_date'] - df['trans_date_max']).dt.days
    df['last_balance_l'] = df['balance_last'] / df['loan_amount']
    df.loc[df['last_balance_l'] == np.inf, 'last_balance_l'] = 0
    df['max_balance_l'] = df['balance_max'] / df['loan_amount']
    df.loc[df['max_balance_l'] == np.inf, 'max_balance_l'] = 0
    df['age_months'] = df['trans_date_days_between'] / 30
    df['bal_per_month'] = df['balance_bal_range'] / df['age_months']
    df['trans_per_month'] = df['operation_count'] / df['age_months']
    df['owner_age_at'] = (df['loan_date'] - df['birth_number']).astype('<m8[Y]') # age in years
    df['owner_age_at'] = df['owner_age_at'].astype(int)
    df['account_age'] = ((df['loan_date'] - df['date']).dt.days) / 30

    return df


train_df = agg_features(train_df)
test_df = agg_features(test_df)
train_df.head()


,loan_id,account_id,loan_date,loan_amount,duration,payments,status,birth_number,district_id,gender,...,trans_type_std_withdrawal,trans_type_std_credit,days_last_trans,last_balance_l,max_balance_l,age_months,bal_per_month,trans_per_month,owner_age_at,account_age
0,4959,2,1994-01-05,80952,24,3373,1,1945-02-04,1,1,...,0.491352,0.491352,5,0.344095,0.834193,10.266667,6470.415584,5.259740,48,10.433333
1,4961,19,1996-04-29,30276,12,2523,-1,1939-04-23,21,0,...,0.494343,0.494343,19,0.523649,1.920911,12.300000,4670.121951,6.504065,57,12.933333
2,4973,67,1996-05-02,165960,24,6915,1,1944-06-13,16,0,...,0.456491,0.456491,2,0.142828,0.645153,18.633333,5708.565295,6.708408,51,18.700000
3,4996,132,1996-11-06,88440,12,7370,1,1945-07-03,40,0,...,0.499740,0.499740,6,0.893347,1.167334,5.766667,17868.034682,5.375723,51,5.966667
4,5002,173,1994-05-31,104808,12,8734,1,1939-11-30,66,1,...,0.489898,0.489898,6,0.267302,0.552108,6.000000,9560.883333,5.000000,54,6.200000


In [8]:
train_df.shape

(328, 73)

Now we need to save the test and train datasets before scaling because these tables are useful for the descriptive task.

In [9]:
train_df.to_csv('train_unsca.csv', index=False)
test_df.to_csv('test_unsca.csv', index=False)

## Encoding and Scaling

For this scenario we encoded all strings and dates using a LabelEncoder which assigns a number to a class.

In [10]:
from sklearn.preprocessing import LabelEncoder

def encode_df(df):
    le = LabelEncoder()
    for col, col_type in df.dtypes.items():
        if col_type == 'object' or col_type == 'datetime64[ns]':
            df[col] = le.fit_transform(df[col])
    return df

train_df = encode_df(train_df)
test_df = encode_df(test_df)

Data was scaled using a QuantileTransformer which transforms data to follow a normal distribution.

In [11]:
from sklearn.preprocessing import QuantileTransformer

scaler = QuantileTransformer(n_quantiles=100, random_state=1, output_distribution='normal')

# don't scale id columns or the status
cols = [col for col in train_df.columns if col != 'loan_id' and col != 'status']
train_df[cols] = scaler.fit_transform(train_df[cols])
test_df[cols] = scaler.fit_transform(test_df[cols])

train_df['status'] = train_df['status'].astype(int)
test_df['status'] = test_df['status'].astype(int)

train_df.head()

,loan_id,account_id,loan_date,loan_amount,duration,payments,status,birth_number,district_id,gender,...,trans_type_std_withdrawal,trans_type_std_credit,days_last_trans,last_balance_l,max_balance_l,age_months,bal_per_month,trans_per_month,owner_age_at,account_age
0,4959,-5.199338,-1.543098,-0.430727,-0.530220,-0.216904,1,-0.952023,-5.199338,5.199338,...,0.063341,0.063341,0.178175,0.111650,0.236579,-0.249863,-0.187886,-0.268923,0.816627,-0.269066
1,4961,-3.034250,0.604585,-1.335178,-5.199338,-0.622082,-1,-1.619856,-0.458679,-5.199338,...,0.321971,0.321971,2.455101,0.544529,1.112094,0.049948,-0.808732,0.553766,1.807354,0.114185
2,4973,-2.603792,0.619855,0.356532,-0.530220,1.075550,1,-1.024053,-0.604585,-5.199338,...,-1.119968,-1.119968,-0.781781,-0.935819,-0.112402,1.029957,-0.425841,0.651081,1.029957,1.029957
3,4996,-2.356668,1.544916,-0.301747,-5.199338,1.367558,1,-0.902159,0.012660,-5.199338,...,1.399657,1.399657,0.403108,0.925573,0.516890,-1.025023,1.439869,-0.166433,1.029957,-1.021681
4,5002,-2.272299,-1.006949,-0.067424,-5.199338,2.301079,1,-1.496373,0.987837,5.199338,...,-0.048867,-0.048867,0.403108,-0.213490,-0.265617,-0.987837,0.381108,-0.544847,1.399657,-0.979545


In [12]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)